In [1]:
import boa
from utils import deploy_infra

In [2]:
# deploying infra for twocrypto-ng pools
infra = deploy_infra()
factory = infra["factory"]
admin = infra["admin"]
coin_a = infra["coin_a"]
coin_b = infra["coin_b"]

/Users/mo/Documents/CurveDocs/boa/lib/python3.12/site-packages/vyper/ast/folding.py:153: UserWarning: ZERO_ADDRESS is deprecated. Please use `empty(address)` instead.
  warnings.warn(f"{name} is deprecated. Please use `{replacement}` instead.")
/Users/mo/Documents/CurveDocs/boa/lib/python3.12/site-packages/vyper/semantics/types/subscriptable.py:115: UserWarning: Use of large arrays can be unsafe!
  warnings.warn("Use of large arrays can be unsafe!")


## **Setting New Implementations**

New implementations can only be set by the `admin` of the Factory. While there can be multiple implementations for pools, there can only be one for gauge, views, and math implementations.

- `set_pool_implementation(_pool_implementation: address, _implementation_index: uint256):`
- `set_gauge_implementation(_gauge_implementation: address):`
- `set_views_implementation(_views_implementation: address):`
- `set_math_implementation(_math_implementation: address):`

In [3]:
# we check the current implementations (these implementations are generated in utils.py)

print("CURRENT IMPLEMENTATIONS")
print("pool implementation:", factory.pool_implementations(0))
print("gauge implementation:", factory.gauge_implementation())
print("views implementation:", factory.views_implementation())
print("math implementation:", factory.math_implementation())

CURRENT IMPLEMENTATIONS
pool implementation: 0x3d06E92f20305D9a2D71a1D479E9EE22690Ae7E4
gauge implementation: 0x78548820b365886d05009F1127bf553603E5A836
views implementation: 0xDa5e407C7b1887E7f76c920B70614e73feA0DDA1
math implementation: 0xB822167C7EefF0B53DcfDEE2D8fe73dEDB25505b


In [4]:
# we set new implementations

with boa.env.prank(admin):
    factory.set_pool_implementation("0x0000000000000000000000000000000000000000", 0)
    factory.set_gauge_implementation("0x0000000000000000000000000000000000000000")
    factory.set_views_implementation("0x0000000000000000000000000000000000000000")
    factory.set_math_implementation("0x0000000000000000000000000000000000000000")


print("NEW IMPLEMENTATIONS")
print("pool implementation:", factory.pool_implementations(0))
print("gauge implementation:", factory.gauge_implementation())
print("views implementation:", factory.views_implementation())
print("math implementation:", factory.math_implementation())

NEW IMPLEMENTATIONS
pool implementation: 0x0000000000000000000000000000000000000000
gauge implementation: 0x0000000000000000000000000000000000000000
views implementation: 0x0000000000000000000000000000000000000000
math implementation: 0x0000000000000000000000000000000000000000


---

## **Changing the Fee Receiver**

The `fee_receiver` is the address which receives the admin fee from all deployed pools from this Factory.

- `Factory.set_fee_receiver(_fee_receiver: address):`"

    Function to set a new fee receiver address.

    Emits: `UpdateFeeReceiver`

    | Input              | Type      | Description                  |
    | ------------------ | --------- | ---------------------------- |
    | `_fee_receiver`    | `uint256` | New fee receiver address     |

In [5]:
print("fee receiver before change:", factory.fee_receiver())

with boa.env.prank(admin):
    factory.set_fee_receiver("0x0000000000000000000000000000000000000000")

print("new fee receiver:", factory.fee_receiver())

fee receiver before change: 0xA73d7cddCf77c00827459f986bf828999B58C6Fe
new fee receiver: 0x0000000000000000000000000000000000000000


---

## **Changing Contract Ownership**


- ### `Factory.commit_transfer_ownership(_addr: address):`

    Function to commit the transfer of ownership. The function sets `future_admin` to the input address `_addr`.

    | Input      | Type      | Description   |
    | ---------- | --------- | ------------- |
    | `_addr`    | `address` | New admin address |

- ### `Factory.accept_transfer_ownership():`

    Function to accept the ownership transfer by the future admin.

    Emits: `TransferOwnership`

*Ownership can be changed with the following flow:*

1. `commit_transfer_ownership`: This call sets the `future_admin` to the input value used in the function. This function can only be called by the current `admin`.
2. `accept_transfer_ownership`: After the `future_admin` is set, these changes need to be accepted by the future admin. After the changes are accepted, the `future_admin` is not reset to `ZERO_ADDRESS`, but rather stays like this until it's changed again.

In [6]:
new_admin = "0x40907540d8a6C65c637785e8f8B742ae6b0b9968"

print("BEFORE ANY CHANGES")
print("current admin:", factory.admin())
print("current future_admin:", factory.future_admin())
print("-----------------")

# admin commits changes
with boa.env.prank(admin):
    factory.commit_transfer_ownership(new_admin)

print("AFTER COMMITING/BEFORE ACCEPTING THE CHANGES")
print("current admin:", factory.admin())
print("current future_admin:", factory.future_admin())
print("-----------------")

# future_admin accepts changes
with boa.env.prank(new_admin):
    factory.accept_transfer_ownership()

print("AFTER ACCEPTING THE CHANGES")
print("current admin:", factory.admin())
print("current future_admin:", factory.future_admin())


BEFORE ANY CHANGES
current admin: 0xd13f0Bd22AFF8176761AEFBfC052a7490bDe268E
current future_admin: 0x0000000000000000000000000000000000000000
-----------------
AFTER COMMITING/BEFORE ACCEPTING
current admin: 0xd13f0Bd22AFF8176761AEFBfC052a7490bDe268E
current future_admin: 0x40907540d8a6C65c637785e8f8B742ae6b0b9968
-----------------
AFTER ACCEPTING THE CHANGES
current admin: 0x40907540d8a6C65c637785e8f8B742ae6b0b9968
current future_admin: 0x40907540d8a6C65c637785e8f8B742ae6b0b9968
